<a href="https://colab.research.google.com/github/prasanth5reddy/D2L/blob/master/Linear%20Neural%20Networks/concise_implementation_of_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Libraries

In [1]:
!pip install mxnet

Importing Libraries

In [0]:
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import data as gdata, nn, loss as gloss

Generating Data Sets

In [0]:
num_inputs = 2
num_examples = 1000
true_w = nd.array([2, -3.4])
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = nd.dot(features, true_w) + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

Reading Data

In [0]:
batch_size = 10

# Combine the features and labels of the training data
dataset = gdata.ArrayDataset(features, labels)
# Randomly reading mini-batches
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [5]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.52217364 -0.96701854]
 [-0.67941266 -0.15194277]
 [ 1.2163423   0.4857486 ]
 [-1.3725436   0.0290854 ]
 [ 1.4929712  -1.7285388 ]
 [-0.42707378  1.4961599 ]
 [-0.86253977 -0.9446958 ]
 [ 0.78226787 -0.6742348 ]
 [-0.90053743 -0.1930618 ]
 [-0.52853745 -1.88909   ]]
<NDArray 10x2 @cpu(0)> 
[ 8.526825   3.3611004  4.9856253  1.3643551 13.052138  -1.7333063
  5.693961   8.047689   3.0568447  9.5670185]
<NDArray 10 @cpu(0)>


Define the model

In [0]:
net = nn.Sequential()
net.add(nn.Dense(1))

Initialize Model Parameters

In [0]:
net.initialize(init.Normal(sigma=0.01))
# updates are deferred until the first time that data is sent through the networks
# Hence parameters won't be initialized until it sees the data

Define the Loss Functon

In [0]:
loss = gloss.L2Loss()

Define the Optimization Algorithm

In [0]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

Training

In [10]:
num_epochs = 3

for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.040816
epoch 2, loss: 0.000155
epoch 3, loss: 0.000050


In [11]:
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)

Error in estimating w 
[[ 0.00032294 -0.00033808]]
<NDArray 1x2 @cpu(0)>
Error in estimating b 
[0.0004549]
<NDArray 1 @cpu(0)>
